In [236]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.datasets import make_regression, make_friedman1
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

x, y = make_friedman1(n_features=500, noise=0.5, n_samples=1000)

In [237]:
reg = LinearRegression()
reg.fit(x, y)
sklearn_mse = mean_squared_error(y, reg.predict(x))

In [238]:
class MSELoss(object):
    """Mean Squared Error loss function"""
    
    def get_neg_gradient(self, y_true, y):
        return y_true - y
    
    def __call__(self, y_true, y):
        return np.mean((y_true - y) ** 2)
    
    
loss = MSELoss()
my_mse = loss(y, reg.predict(x))

my_mse == sklearn_mse

True

In [298]:
from scipy.optimize import minimize
from sklearn.tree import DecisionTreeRegressor

class GradientBoostingRegressor(object):
    """Simple Gradient Boosting Regressor implementation"""
    def __init__(self, weak_learner, learning_rate, num_rounds, subsample=1.0, adaptive_learning_rate=False, weak_learner_params_dict={}):
        """Parameters
        -------------
        weak_learner: sklearn regressor
            a weak learner to boost with
        learning_rate: float
            a learning rate regularization coefficient, used as initial guess is used with 
            adaptive_learning_rate mode
        num_rounds: int
            number of boosting rounds
        subsample: float
            subsample provideded percentage of observations training set in each boosting 
            round in the spirit of bagging
        adaptive_learning_rate: bool
            try to find an optimal learning rate for each boosting round, learning_rate aprameter 
            is used as an initial guess in this case
        weak_learner_params_dict: dict
            any additional parameters to be passed to the weak_learner
        """
        self.learning_rate = learning_rate
        self.num_rounds = num_rounds
        self.weak_learner = weak_learner
        self.weak_learner_params_dict = weak_learner_params_dict
        self.adaptive_learning_rate = adaptive_learning_rate
        self._loss = MSELoss()
        self._gammas = [1]
        self._estimators = []
        self._subsample_rate = subsample
        
    def fit(self, x, y):
        """Fit a Gradient Boosting Regressor"""
        self._fit_base_model(x, y)
        preds = self._base_model.predict(x)
        
        for i in range(0, self.num_rounds):
            preds += self._boosting_round(x, y, preds)
            
    def predict(self, x):
        """Predict using a trained model"""
        if len(self._estimators) == 0:
            raise Exception("The model should be trained first. \
                            Please, call the fit method before generating predictions")
            
        result = self._estimators[0].predict(x)
        for i, estimator in enumerate(self._estimators[1:]):
            result += estimator.predict(x) * self._gammas[i + 1]
            
        return result
    
    def _boosting_round(self, x, y, preds):
        """Performs a single gradient boosting round"""
        x_ss, y_ss, preds_ss = self._subsample(self._subsample_rate, x, y, preds)
        residual = self._loss.get_neg_gradient(y_ss, preds_ss)
        
        boosting_model = self.weak_learner(**self.weak_learner_params_dict)
        boosting_model.fit(x_ss, residual)
        old_preds = preds
        preds = boosting_model.predict(x)
        
        if self.adaptive_learning_rate:
            gamma = minimize(self._gamma_obj, 
                             self.learning_rate, 
                             args=(y, old_preds, preds), 
                             tol=1e-28, 
                             method = 'Nelder-Mead').x[0]
        else:
            gamma = self.learning_rate
        
        self._estimators.append(boosting_model)
        self._gammas.append(gamma)
        
        return preds * gamma
    
    def _fit_base_model(self, x, y):
        """Fits a base weak learner to start boosting with"""
        self._base_model = self.weak_learner(**self.weak_learner_params_dict)
        self._base_model.fit(x, y)
        self._estimators.append(self._base_model)
        
    @staticmethod
    def _gamma_obj(gamma, y, old_preds, preds):
        """Objective function used to minimize learning rate when using adaptive mode"""
        return self.loss(y, old_preds + gamma * preds)
    
    @staticmethod
    def _subsample(subsample_rate, x, y, preds):
        """Generates a random subsample with replacement"""
        sample_size = max(1, subsample_rate * len(x))
        mask = np.random.randint(len(x), size=sample_size)
        return x[mask, :], y[mask], preds[mask]
            
gbm = GradientBoostingRegressor(DecisionTreeRegressor, 
                                learning_rate=0.05, 
                                num_rounds=100, 
                                adaptive_learning_rate=False,
                                weak_learner_params_dict = {
                                    'max_depth': 20
                                })
gbm.fit(x, y)
p = gbm.predict(x)
my_mse, loss(y, p)

/home/kdubovikov/opt/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:93: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


(2.9560358708320256, 2.0291945635469445e-08)